In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 2.7MB/s 


In [3]:
import pandas as pd
import sys
sys.path.insert(0,'/content/drive/My Drive/Datasets_SVR/Library')
from subsamplesvr import svr_subsample_bayes_train
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.svm import SVR
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV

In [4]:
data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/PRSA_data_2010.1.1-2014.12.31 (1).csv")

In [5]:
data.describe()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir
count,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,41757.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000
mean,21912.500000,2012.000000,6.523549,15.727820,11.500000,98.613215,1.817246,12.448521,1016.447654,23.889140,0.052734,0.194916
std,12651.043435,1.413842,3.448572,8.799425,6.922266,92.050387,14.433440,12.198613,10.268698,50.010635,0.760375,1.415867
min,1.000000,2010.000000,1.000000,1.000000,0.000000,0.000000,-40.000000,-19.000000,991.000000,0.450000,0.000000,0.000000
25%,10956.750000,2011.000000,4.000000,8.000000,5.750000,29.000000,-10.000000,2.000000,1008.000000,1.790000,0.000000,0.000000
50%,21912.500000,2012.000000,7.000000,16.000000,11.500000,72.000000,2.000000,14.000000,1016.000000,5.370000,0.000000,0.000000
75%,32868.250000,2013.000000,10.000000,23.000000,17.250000,137.000000,15.000000,23.000000,1025.000000,21.910000,0.000000,0.000000
max,43824.000000,2014.000000,12.000000,31.000000,23.000000,994.000000,28.000000,42.000000,1046.000000,585.600000,27.000000,36.000000


In [6]:
print(data.shape)

(43824, 13)


In [7]:
data.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [8]:
data.isna().sum()

No          0
year        0
month       0
day         0
hour        0
pm2.5    2067
DEWP        0
TEMP        0
PRES        0
cbwd        0
Iws         0
Is          0
Ir          0
dtype: int64

In [9]:
data=data.drop(data.columns[[0]], axis='columns')

In [10]:
data

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43819,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [11]:
data=data.dropna()

In [12]:
data.isna().sum()

year     0
month    0
day      0
hour     0
pm2.5    0
DEWP     0
TEMP     0
PRES     0
cbwd     0
Iws      0
Is       0
Ir       0
dtype: int64

In [13]:
data.shape

(41757, 12)

In [14]:
data.reset_index(drop=True,inplace=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41757 entries, 0 to 41756
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    41757 non-null  int64  
 1   month   41757 non-null  int64  
 2   day     41757 non-null  int64  
 3   hour    41757 non-null  int64  
 4   pm2.5   41757 non-null  float64
 5   DEWP    41757 non-null  int64  
 6   TEMP    41757 non-null  float64
 7   PRES    41757 non-null  float64
 8   cbwd    41757 non-null  object 
 9   Iws     41757 non-null  float64
 10  Is      41757 non-null  int64  
 11  Ir      41757 non-null  int64  
dtypes: float64(4), int64(7), object(1)
memory usage: 3.8+ MB


In [16]:
data.cbwd.value_counts()

SE    14573
NW    13484
cv     8944
NE     4756
Name: cbwd, dtype: int64

In [17]:
data['cbwd'].replace(to_replace=['SE'],value=0,inplace=True)
data['cbwd'].replace(to_replace=['NW'],value=1,inplace=True)
data['cbwd'].replace(to_replace=['cv'],value=2,inplace=True)
data['cbwd'].replace(to_replace=['NE'],value=3,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [18]:
data.cbwd.value_counts()

0    14573
1    13484
2     8944
3     4756
Name: cbwd, dtype: int64

In [19]:
data

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,2010,1,2,0,129.0,-16,-4.0,1020.0,0,1.79,0,0
1,2010,1,2,1,148.0,-15,-4.0,1020.0,0,2.68,0,0
2,2010,1,2,2,159.0,-11,-5.0,1021.0,0,3.57,0,0
3,2010,1,2,3,181.0,-7,-5.0,1022.0,0,5.36,1,0
4,2010,1,2,4,138.0,-7,-5.0,1022.0,0,6.25,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
41752,2014,12,31,19,8.0,-23,-2.0,1034.0,1,231.97,0,0
41753,2014,12,31,20,10.0,-22,-3.0,1034.0,1,237.78,0,0
41754,2014,12,31,21,10.0,-22,-3.0,1034.0,1,242.70,0,0
41755,2014,12,31,22,8.0,-22,-4.0,1034.0,1,246.72,0,0


In [20]:
data['year'] = data['year'].astype(str)
data['month'] = data['month'].astype(str)
data['day'] = data['day'].astype(str)
data['hour'] = data['hour'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41757 entries, 0 to 41756
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    41757 non-null  object 
 1   month   41757 non-null  object 
 2   day     41757 non-null  object 
 3   hour    41757 non-null  object 
 4   pm2.5   41757 non-null  float64
 5   DEWP    41757 non-null  int64  
 6   TEMP    41757 non-null  float64
 7   PRES    41757 non-null  float64
 8   cbwd    41757 non-null  int64  
 9   Iws     41757 non-null  float64
 10  Is      41757 non-null  int64  
 11  Ir      41757 non-null  int64  
dtypes: float64(4), int64(4), object(4)
memory usage: 3.8+ MB


In [22]:
data["Date"]=data['year']+"/"+data['month']+"/"+data['day']+" "+data['hour']+":00:00"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
data

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,Date
0,2010,1,2,0,129.0,-16,-4.0,1020.0,0,1.79,0,0,2010/1/2 0:00:00
1,2010,1,2,1,148.0,-15,-4.0,1020.0,0,2.68,0,0,2010/1/2 1:00:00
2,2010,1,2,2,159.0,-11,-5.0,1021.0,0,3.57,0,0,2010/1/2 2:00:00
3,2010,1,2,3,181.0,-7,-5.0,1022.0,0,5.36,1,0,2010/1/2 3:00:00
4,2010,1,2,4,138.0,-7,-5.0,1022.0,0,6.25,2,0,2010/1/2 4:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41752,2014,12,31,19,8.0,-23,-2.0,1034.0,1,231.97,0,0,2014/12/31 19:00:00
41753,2014,12,31,20,10.0,-22,-3.0,1034.0,1,237.78,0,0,2014/12/31 20:00:00
41754,2014,12,31,21,10.0,-22,-3.0,1034.0,1,242.70,0,0,2014/12/31 21:00:00
41755,2014,12,31,22,8.0,-22,-4.0,1034.0,1,246.72,0,0,2014/12/31 22:00:00


In [24]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d %H:%M:%S')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41757 entries, 0 to 41756
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   year    41757 non-null  object        
 1   month   41757 non-null  object        
 2   day     41757 non-null  object        
 3   hour    41757 non-null  object        
 4   pm2.5   41757 non-null  float64       
 5   DEWP    41757 non-null  int64         
 6   TEMP    41757 non-null  float64       
 7   PRES    41757 non-null  float64       
 8   cbwd    41757 non-null  int64         
 9   Iws     41757 non-null  float64       
 10  Is      41757 non-null  int64         
 11  Ir      41757 non-null  int64         
 12  Date    41757 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(4)
memory usage: 4.1+ MB


In [26]:
data.drop(data.columns[[0,1,2,3]],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
data

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,Date
0,129.0,-16,-4.0,1020.0,0,1.79,0,0,2010-01-02 00:00:00
1,148.0,-15,-4.0,1020.0,0,2.68,0,0,2010-01-02 01:00:00
2,159.0,-11,-5.0,1021.0,0,3.57,0,0,2010-01-02 02:00:00
3,181.0,-7,-5.0,1022.0,0,5.36,1,0,2010-01-02 03:00:00
4,138.0,-7,-5.0,1022.0,0,6.25,2,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...
41752,8.0,-23,-2.0,1034.0,1,231.97,0,0,2014-12-31 19:00:00
41753,10.0,-22,-3.0,1034.0,1,237.78,0,0,2014-12-31 20:00:00
41754,10.0,-22,-3.0,1034.0,1,242.70,0,0,2014-12-31 21:00:00
41755,8.0,-22,-4.0,1034.0,1,246.72,0,0,2014-12-31 22:00:00


In [28]:
data.sort_values(by='Date',inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
data

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,Date
0,129.0,-16,-4.0,1020.0,0,1.79,0,0,2010-01-02 00:00:00
1,148.0,-15,-4.0,1020.0,0,2.68,0,0,2010-01-02 01:00:00
2,159.0,-11,-5.0,1021.0,0,3.57,0,0,2010-01-02 02:00:00
3,181.0,-7,-5.0,1022.0,0,5.36,1,0,2010-01-02 03:00:00
4,138.0,-7,-5.0,1022.0,0,6.25,2,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...
41752,8.0,-23,-2.0,1034.0,1,231.97,0,0,2014-12-31 19:00:00
41753,10.0,-22,-3.0,1034.0,1,237.78,0,0,2014-12-31 20:00:00
41754,10.0,-22,-3.0,1034.0,1,242.70,0,0,2014-12-31 21:00:00
41755,8.0,-22,-4.0,1034.0,1,246.72,0,0,2014-12-31 22:00:00


In [30]:
data['Date'].min()

Timestamp('2010-01-02 00:00:00')

In [31]:
data['Date'].max()

Timestamp('2014-12-31 23:00:00')

In [32]:
data.drop('Date',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
data

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,129.0,-16,-4.0,1020.0,0,1.79,0,0
1,148.0,-15,-4.0,1020.0,0,2.68,0,0
2,159.0,-11,-5.0,1021.0,0,3.57,0,0
3,181.0,-7,-5.0,1022.0,0,5.36,1,0
4,138.0,-7,-5.0,1022.0,0,6.25,2,0
...,...,...,...,...,...,...,...,...
41752,8.0,-23,-2.0,1034.0,1,231.97,0,0
41753,10.0,-22,-3.0,1034.0,1,237.78,0,0
41754,10.0,-22,-3.0,1034.0,1,242.70,0,0
41755,8.0,-22,-4.0,1034.0,1,246.72,0,0


In [34]:
data.describe()

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
count,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000
mean,98.613215,1.750174,12.401561,1016.442896,1.092990,23.866747,0.055344,0.194866
std,92.050387,14.433658,12.175215,10.300733,1.005063,49.617495,0.778875,1.418165
min,0.000000,-40.000000,-19.000000,991.000000,0.000000,0.450000,0.000000,0.000000
25%,29.000000,-10.000000,2.000000,1008.000000,0.000000,1.790000,0.000000,0.000000
50%,72.000000,2.000000,14.000000,1016.000000,1.000000,5.370000,0.000000,0.000000
75%,137.000000,15.000000,23.000000,1025.000000,2.000000,21.910000,0.000000,0.000000
max,994.000000,28.000000,42.000000,1046.000000,3.000000,565.490000,27.000000,36.000000


In [35]:
train_size=int(data.shape[0]*0.8)

In [36]:
train_size

33405

In [37]:
x_train=data.iloc[:train_size]

In [38]:
x_train

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,129.0,-16,-4.0,1020.0,0,1.79,0,0
1,148.0,-15,-4.0,1020.0,0,2.68,0,0
2,159.0,-11,-5.0,1021.0,0,3.57,0,0
3,181.0,-7,-5.0,1022.0,0,5.36,1,0
4,138.0,-7,-5.0,1022.0,0,6.25,2,0
...,...,...,...,...,...,...,...,...
33400,177.0,-18,-7.0,1031.0,3,1.79,0,0
33401,287.0,-17,-7.0,1032.0,0,0.89,0,0
33402,299.0,-19,-8.0,1032.0,3,1.79,0,0
33403,250.0,-18,-7.0,1033.0,1,1.79,0,0


In [39]:
x_test=data.iloc[train_size:]

In [40]:
x_test

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
33405,124.0,-20,-9.0,1033.0,1,6.71,0,0
33406,123.0,-19,-8.0,1034.0,1,11.63,0,0
33407,49.0,-21,-10.0,1034.0,1,15.65,0,0
33408,36.0,-19,-9.0,1034.0,2,0.89,0,0
33409,59.0,-20,-9.0,1034.0,2,1.78,0,0
...,...,...,...,...,...,...,...,...
41752,8.0,-23,-2.0,1034.0,1,231.97,0,0
41753,10.0,-22,-3.0,1034.0,1,237.78,0,0
41754,10.0,-22,-3.0,1034.0,1,242.70,0,0
41755,8.0,-22,-4.0,1034.0,1,246.72,0,0


In [41]:
from sklearn.utils import shuffle

In [42]:
x_train=shuffle(x_train)

In [43]:
x_test=shuffle(x_test)

In [44]:
x_train

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
15942,27.0,-17,3.0,1028.0,1,4.92,0,0
15733,11.0,-21,-2.0,1036.0,1,80.02,0,0
2588,73.0,-2,7.0,1019.0,0,35.77,0,0
20185,132.0,20,24.0,1002.0,0,33.97,0,0
26246,10.0,-22,13.0,1008.0,1,4.02,0,0
...,...,...,...,...,...,...,...,...
28563,57.0,19,21.0,1003.0,2,0.89,0,0
1712,41.0,-11,2.0,1026.0,3,9.84,0,0
12682,233.0,26,31.0,1004.0,0,5.81,0,0
19091,76.0,8,16.0,1005.0,2,0.89,0,0


In [45]:
x_test

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
35630,145.0,0,14.0,1010.0,0,8.05,0,1
38114,144.0,22,27.0,1005.0,0,20.13,0,0
33650,231.0,-12,-1.0,1017.0,1,1.79,0,0
40294,159.0,8,10.0,1023.0,2,0.90,0,0
38810,78.0,18,19.0,1015.0,1,1.79,0,0
...,...,...,...,...,...,...,...,...
40936,289.0,0,7.0,1015.0,2,1.78,0,0
36532,79.0,18,20.0,1008.0,0,14.31,0,0
38598,73.0,19,25.0,1010.0,0,40.67,0,0
40245,77.0,1,3.0,1029.0,3,0.89,0,0


In [46]:
col_names=list(data.columns[data.columns!="pm2.5"])

In [47]:
col_names

['DEWP', 'TEMP', 'PRES', 'cbwd', 'Iws', 'Is', 'Ir']

In [48]:
scaler = StandardScaler()
scaled_data=scaler.fit_transform(x_train.loc[:,x_train.columns!="pm2.5"])

In [49]:
x_train[col_names]=scaled_data

In [50]:
scaler = StandardScaler()
scaled_data=scaler.fit_transform(x_test.loc[:,x_test.columns!="pm2.5"])

In [51]:
x_test[col_names]=scaled_data

In [52]:
x_train

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
15942,27.0,-1.271166,-0.725545,1.100592,-0.092972,-0.393536,-0.07639,-0.13993
15733,11.0,-1.546008,-1.131555,1.865822,-0.092972,1.085352,-0.07639,-0.13993
2588,73.0,-0.240508,-0.400737,0.239708,-1.094501,0.213970,-0.07639,-0.13993
20185,132.0,1.271123,0.979696,-1.386405,-1.094501,0.178524,-0.07639,-0.13993
26246,10.0,-1.614718,0.086474,-0.812483,-0.092972,-0.411259,-0.07639,-0.13993
...,...,...,...,...,...,...,...,...
28563,57.0,1.202412,0.736090,-1.290751,0.908556,-0.472896,-0.07639,-0.13993
1712,41.0,-0.858903,-0.806747,0.909285,1.910084,-0.296650,-0.07639,-0.13993
12682,233.0,1.683386,1.548110,-1.195097,-1.094501,-0.376010,-0.07639,-0.13993
19091,76.0,0.446597,0.330080,-1.099444,0.908556,-0.472896,-0.07639,-0.13993


In [53]:
x_test

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
35630,145.0,-0.197850,-0.023433,-0.645911,-1.060811,-0.262677,-0.047879,0.818894
38114,144.0,1.385228,1.115968,-1.163592,-1.060811,0.009303,-0.047879,-0.130496
33650,231.0,-1.061347,-1.338127,0.078842,-0.090820,-0.403620,-0.047879,-0.130496
40294,159.0,0.377814,-0.374018,0.700059,0.879170,-0.423659,-0.047879,-0.130496
38810,78.0,1.097395,0.414798,-0.128230,-0.090820,-0.403620,-0.047879,-0.130496
...,...,...,...,...,...,...,...,...
40936,289.0,-0.197850,-0.636957,-0.128230,0.879170,-0.403846,-0.047879,-0.130496
36532,79.0,1.097395,0.502444,-0.852984,-1.060811,-0.121734,-0.047879,-0.130496
38598,73.0,1.169354,0.940675,-0.645911,-1.060811,0.471759,-0.047879,-0.130496
40245,77.0,-0.125892,-0.987542,1.321277,1.849161,-0.423884,-0.047879,-0.130496


## Baseline for comparison...

In [60]:
start = time.time()
base_rbf=SVR(kernel='rbf').fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred_rbf = base_rbf.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred_rbf))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred_rbf)))
print("SCORE: ",base_rbf.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

MAE:  53.313499002266106
RMSE:  82.67360600098715
SCORE:  0.2310621986300052
total time: 76.59928226470947


In [62]:
start = time.time()
base_linear=SVR(kernel='linear').fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred_linear = base_linear.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred_linear))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred_linear)))
print("SCORE: ",base_linear.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

MAE:  57.739236933936226
RMSE:  87.72277727444347
SCORE:  0.13427054153844187
total time: 52.94134855270386


In [63]:
start = time.time()
base_poly=SVR(kernel='poly').fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred_poly = base_poly.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred_poly))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred_poly)))
print("SCORE: ",base_poly.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

MAE:  58.5489631879306
RMSE:  90.52415331662606
SCORE:  0.07809452756095259
total time: 77.27744817733765


In [64]:
start = time.time()
base_sig=SVR(kernel='sigmoid').fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred_sig = base_sig.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred_sig))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred_sig)))
print("SCORE: ",base_sig.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

MAE:  211.10497610717482
RMSE:  319.34501950491915
SCORE:  -10.473019839167588
total time: 108.78591108322144


## SVR subsmple with Bayesian Optimization

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"pm2.5","rbf",5,0.30)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: -0.08034678995000373
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: 0.12378940763506538
Interrupting!
++++++ TRAIN TIME (0) 0.11417102813720703 +++++++++++
####### BEST MODEL PARAMS INITIAL STEP (0) #####
OrderedDict([('C', 10.610876419084347), ('degree', 3), ('epsilon', 0.026340889765441144), ('kernel', 'rbf')])
Coefficient of determination R^2 of the prediction: 



[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


0.08889637554541874
MAE:  57.132152497902226
RMSE:  89.99226075925397
####### BEST MODEL PARAMS INITIAL STEP (0) #####
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.26642875168412206
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.3s finished


best score: 0.36246634705739766
Interrupting!
++++++ TRAIN TIME (1) 4.04476523399353 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 435.6046845781492), ('degree', 4), ('epsilon', 39.069242441028905), ('kernel', 'rbf')])
Coefficient of determination R^2 of the prediction: 

0.30294695310967934
MAE:  54.550486190722985
RMSE:  78.71440010328739
DIFF with previous:  -0.2140505775642606
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 6.916952610015869 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=10, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['rbf']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"pm2.5","linear",5,0.20)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: 0.20280798789831816
Interrupting!
++++++ TRAIN TIME (0) 0.12490987777709961 +++++++++++
####### BEST MODEL PARAMS INITIAL STEP (0) #####
OrderedDict([('C', 265.5400346806723), ('degree', 3), ('epsilon', 0.018226270497287), ('kernel', 'linear')])
Coefficient of determination R^2 of the prediction: 

0.12841140168343967
MAE:  57.60490209279774
RMSE:  88.01912465473463
####### BEST MODEL PARAMS INITIAL STEP (0) #####


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.1884120831618662
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    2.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.19166828142557588
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    8.2s finished


best score: 0.2002502566536006
Interrupting!
++++++ TRAIN TIME (1) 30.985352039337158 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 423.9363679186768), ('degree', 4), ('epsilon', 10.62715350792161), ('kernel', 'linear')])
Coefficient of determination R^2 of the prediction: 

0.15747301569553462
MAE:  57.837159032149835
RMSE:  86.53926148315364
DIFF with previous:  -0.02906161401209495
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 33.25763010978699 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=10, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['linear']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.1,0.1,"pm2.5","sigmoid",5,0.30)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    2.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: -0.03082755169431946
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.008825248157563647
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    5.3s finished


best score: 0.020131845540941752
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    5.3s finished


best score: 0.020131845540941752
++++++ TRAIN TIME (ITER) 53.503153562545776 +++++++++++
####### BEST MODEL PARAMS ITER 0 #####
OrderedDict([('C', 0.040183884803972086), ('degree', 2), ('epsilon', 72.57105670933664), ('kernel', 'sigmoid')])
Coefficient of determination R^2 of the prediction: 

0.01536910705247352
MAE:  69.05833450654657
RMSE:  93.55306142001798
DIFF with previous:  -0.03270333908073986
####### BEST MODEL PARAMS ITER 0 #####
!!!! TOTAL ELAPSED TIME: 262.3578100204468 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=10, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['sigmoid']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"pm2.5","poly",5,0.10)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: -0.2005952066600811
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: -0.2005952066600811
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


best score: -0.2005952066600811
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: -0.005105595396971493
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: -0.005105595396971493
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: -0.005105595396971493
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: -0.005105595396971493
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score: -0.005105595396971493
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    2.3s finished


best score: -0.018876034892962124
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.4s finished


best score: 0.025889781747024127
++++++ TRAIN TIME (1) 58.59252905845642 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 0.8152279711864749), ('degree', 3), ('epsilon', 94.06604186226583), ('kernel', 'poly')])
Coefficient of determination R^2 of the prediction: 

0.023734499577531043
MAE:  71.85483845079234
RMSE:  93.15480180895082
DIFF with previous:  -0.12179723057963843
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 63.36580514907837 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=10, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['poly']},
              verbose=3)

## Comparison without the subsample algorithm

In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['rbf']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    n_iter=10,
    verbose=3
)
# callback handler
def on_step(optim_result):
    score = Sv.best_score_
    print("best score: %s" % score)
    if score >= 0.30:
        print('Interrupting!')
        return True

Sv.fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"],callback=on_step)
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.5min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.11710329761601082
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.8min finished


best score: 0.35625636472662525
Interrupting!
MAE:  50.68663079521919
RMSE:  78.65243678695825
SCORE:  0.3040439498042242
total time: 297.0749623775482


In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['linear']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    n_iter=10,
    verbose=3
)
# callback handler
def on_step(optim_result):
    score = Sv.best_score_
    print("best score: %s" % score)
    if score >= 0.20:
        print('Interrupting!')
        return True

Sv.fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"],callback=on_step)
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   53.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.1270900388391683
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   34.3s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.18383285096056556
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

best score: 0.18383285096056556
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   53.3s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.18383285096056556
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   10.3s finished


best score: 0.20807197361240284
Interrupting!
MAE:  64.28481323451331
RMSE:  84.82756163653643
SCORE:  0.1904728536557765
total time: 463.22995471954346


In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['poly']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    n_iter=10,
    verbose=3
)
# callback handler
def on_step(optim_result):
    score = Sv.best_score_
    print("best score: %s" % score)
    if score >= 0.30:
        print('Interrupting!')
        return True

Sv.fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"],callback=on_step)
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  7.5min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.17448272755988045
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.0min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.17448272755988045
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.8min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.17448272755988045
Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['sigmoid']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    n_iter=10,
    verbose=3
)
# callback handler
def on_step(optim_result):
    score = Sv.best_score_
    print("best score: %s" % score)
    if score >= 0.30:
        print('Interrupting!')
        return True

Sv.fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"],callback=on_step)
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  2.1min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: -1974404.2359920645
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  2.1min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: -8.047267516155728
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

best score: 0.10227122399188633
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  2.0min finished


best score: 0.10227122399188633
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.1min finished


best score: 0.10227122399188633
MAE:  66.6962412444937
RMSE:  94.6022393375681
SCORE:  -0.006839588421699405
total time: 1166.9439601898193
